建立加權指數 個股 連線

In [1]:
import pandas as pd
import pymysql

class MySQLUpdater:
    def __init__(self, db_params, table_name, df_stocks, condition, method):
        self.db = pymysql.connect(**db_params)
        self.cursor = self.db.cursor()
        self.table_name = table_name
        self.df_stocks = df_stocks
        self.condition = condition
        self.method = method
        


    def update_records(self):
        # 遍历 DataFrame 中的每一行

        for i, row in self.df_stocks.iterrows():
            try:
                if(self.method == '1'):
                    # 构建 SQL 更新语句
                    sql = f"""
                    UPDATE `{self.table_name}`
                    SET `5MA` = %s,
                        `8MA` = %s,
                        `10MA` = %s,
                        `20MA` = %s,
                        `20RS` = %s,
                        `60MA` = %s,
                        `60RS` = %s,
                        `240MA` = %s,
                        `240RS` = %s
                    WHERE `{self.condition}` = %s AND `Date` = %s
                    """
                    # 执行更新语句
                    self.cursor.execute(sql, (float(row['5MA']),float(row['8MA']),float(row['10MA']),float(row['20MA']),float(row['20RS']),float(row['60MA']),float(row['60RS']),float(row['240MA']),float(row['240RS']), row[self.condition], row['Date']))

                    # 提交更改
                    self.db.commit()
                    
                elif (self.method == '2'):

                    # 构建 SQL 更新语句 更新標準化RS
                    sql = f"""
                    UPDATE `{self.table_name}`
                    SET `20RS_sd` = %s,
                        `20RS_rank` = %s,
                        `60RS_sd` = %s,
                        `60RS_rank` = %s,
                        `240RS_sd` = %s,
                        `240RS_rank` = %s
                    WHERE `{self.condition}` = %s AND `Date` = %s
                    """

                    # 执行更新语句
                    self.cursor.execute(sql, (float(row['20RS_sd']),float(row['20RS_rank']),float(row['60RS_sd']),float(row['60RS_rank']),float(row['240RS_sd']),float(row['240RS_rank']), row[self.condition], row['Date']))

                    # 提交更改
                    self.db.commit()

                elif (self.method == '3'):

                    # 构建 SQL 更新语句 更新標準化RS
                    sql = f"""
                    UPDATE `{self.table_name}`
                    SET `收盤價` = %s
                    WHERE `{self.condition}` = %s AND `Date` = %s
                    """

                    # 执行更新语句
                    self.cursor.execute(sql, (float(row['收盤價']), row[self.condition], row['Date']))

                    # 提交更改
                    self.db.commit()

            except Exception as e:
                print(f"Error updating row {i}: {e}")
                self.db.rollback()

    def close_connection(self):
        self.cursor.close()
        self.db.close()

# 示例使用
db_params = {
    'host': '127.0.0.1',
    'user': 'root',
    'password': '19970730',
    'database': 'sql_stock'
    # 設置MySQL資料庫連接
}

前處理 因為有些沒有交易張數的話收盤價會變成0
處理本日收盤價為0的資料 查詢本日收盤資料

In [2]:
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime, timedelta

# 設置MySQL資料庫連接
db_user = 'root'
db_password = '19970730'
db_host = '127.0.0.1'
db_name = 'sql_stock'

# 創建資料庫連接引擎
engine = create_engine(f'mysql+mysqlconnector://{db_user}:{db_password}@{db_host}/{db_name}')

# 設置日期範圍
#today = datetime(2024, 5, 25)
today = datetime.now().date()
one_month_ago = today - timedelta(days=60)

# 格式化日期
today_str = today.strftime('%Y%m%d')
one_month_ago = one_month_ago.strftime('%Y%m%d')

# 可自由調整select的區間
# today_str = '20240813'
# one_month_ago = '20210101'


# SQL 查詢 個股
query_stock = f"""
SELECT *
FROM daily_price
WHERE date >= '{one_month_ago}' AND date <= '{today_str}' 
"""

# 使用 pd.read_sql 來執行查詢並讀取數據到 DataFrame

stock_df_raw_pre = pd.read_sql(query_stock, engine)


補齊收盤價為0 (當入無成交時) 如果不是當天跑 要自己改日期
手動要改的看這邊!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [3]:
today = datetime.now().date()
runday_str = today.strftime('%Y-%m-%d')
# 如果不是當天 要另外手動
runday_str = '2025-02-14'

In [4]:
import numpy as np
stock_df_raw_pre['Date'] = pd.to_datetime(stock_df_raw_pre['Date'])
stock_df_today = stock_df_raw_pre[(stock_df_raw_pre['Date'] == runday_str) & (stock_df_raw_pre['收盤價'] == 0)].copy() 


stock_ids = stock_df_today['證券代號'].unique().tolist()

if (len(stock_ids) != 0) :
        for i in stock_ids:
                if((len(i)==4) ): #1. 排除債 ETF等等 2.排除新掛牌資料不到calculate_day天
                        #print(i)
                        # 計算個股 RS
                        temp_df = stock_df_raw_pre[(stock_df_raw_pre['證券代號']==i) & (stock_df_raw_pre['Date'] == runday_str)].copy()
                        temp_df_his = stock_df_raw_pre[(stock_df_raw_pre['證券代號']==i) & (stock_df_raw_pre['Date'] < runday_str)].tail(1).copy()

                        temp_df.loc[:, '收盤價'] = temp_df_his['收盤價'].iloc[0]

                        # 還是直接算但改用replace改掉 inf
                        temp_df.replace([np.inf, -np.inf], 0, inplace=True)     
                        temp_df = temp_df.fillna(0)
                        # 初始化并更新数据库
                        updater = MySQLUpdater(db_params, 'daily_price', temp_df, '證券代號','3')
                        updater.update_records()
        updater.close_connection()


重新讀取各table值 後續計算MA RS rank

In [5]:
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime, timedelta

# 設置MySQL資料庫連接
db_user = 'root'
db_password = '19970730'
db_host = '127.0.0.1'
db_name = 'sql_stock'

# 創建資料庫連接引擎
engine = create_engine(f'mysql+mysqlconnector://{db_user}:{db_password}@{db_host}/{db_name}')

# 設置日期範圍
#today = datetime(2024, 5, 25)
today = datetime.now().date()
one_year_ago = today - timedelta(days=720)

# 格式化日期
today_str = today.strftime('%Y%m%d')
one_year_ago_str = one_year_ago.strftime('%Y%m%d')

# 可自由調整select的區間
# today_str = '20240813'
# one_year_ago_str = '20210101'

# SQL 查詢 加權指數
query_twse = f"""
SELECT *
FROM daily_twse
WHERE date >= '{one_year_ago_str}' AND date <= '{today_str}'
"""

# SQL 查詢 個股
query_stock = f"""
SELECT *
FROM daily_price
WHERE date >= '{one_year_ago_str}' AND date <= '{today_str}'
"""

# SQL 查詢 細產業
query_sub_category = f"""
SELECT *
FROM sub_category
"""

# 使用 pd.read_sql 來執行查詢並讀取數據到 DataFrame
twse_df_raw = pd.read_sql(query_twse, engine)

stock_df_raw = pd.read_sql(query_stock, engine)

sub_category_list = pd.read_sql(query_sub_category, engine)



In [6]:
s300_df = twse_df_raw[twse_df_raw['指數名稱'] == '小型股300指數'].copy()

s300_df_na = s300_df.isna()
s300_df_update = s300_df[s300_df_na.any(axis=1)] #需要被更新的欄位

s300_df.loc[:, '5MA'] = s300_df['價格指數值'].rolling(5).mean()
s300_df.loc[:, '8MA'] = s300_df['價格指數值'].rolling(8).mean()
s300_df.loc[:, '10MA'] = s300_df['價格指數值'].rolling(10).mean()
s300_df.loc[:, '20MA']= s300_df['價格指數值'].rolling(20).mean()
s300_df.loc[:, '60MA']= s300_df['價格指數值'].rolling(60).mean()
s300_df.loc[:, '240MA']= s300_df['價格指數值'].rolling(240).mean()


s300_df.loc[:, '20RS'] = round((s300_df['價格指數值']/s300_df['20MA'])*100, 4)
s300_df.loc[:, '60RS'] = round((s300_df['價格指數值']/s300_df['60MA'])*100, 4)
s300_df.loc[:, '240RS'] = round((s300_df['價格指數值']/s300_df['240MA'])*100, 4)
s300_df = s300_df.fillna(0)
s300_df_new = s300_df.loc[s300_df_update.index]  # 只更新尚未計算的新欄位


# 初始化并更新数据库
updater = MySQLUpdater(db_params, 'daily_twse', s300_df_new, '指數名稱', '1')
updater.update_records()
updater.close_connection()

計算加權指數各均線及RS

In [7]:
twse_df = twse_df_raw[twse_df_raw['指數名稱'] == '發行量加權股價指數'].copy()

twse_df_na = twse_df.isna()
twse_df_update = twse_df[twse_df_na.any(axis=1)] #需要被更新的欄位

twse_df.loc[:, '5MA'] = twse_df['價格指數值'].rolling(5).mean()
twse_df.loc[:, '8MA'] = twse_df['價格指數值'].rolling(8).mean()
twse_df.loc[:, '10MA'] = twse_df['價格指數值'].rolling(10).mean()
twse_df.loc[:, '20MA']= twse_df['價格指數值'].rolling(20).mean()
twse_df.loc[:, '60MA']= twse_df['價格指數值'].rolling(60).mean()
twse_df.loc[:, '240MA']= twse_df['價格指數值'].rolling(240).mean()


twse_df.loc[:, '20RS'] = round((twse_df['價格指數值']/twse_df['20MA'])*100, 4)
twse_df.loc[:, '60RS'] = round((twse_df['價格指數值']/twse_df['60MA'])*100, 4)
twse_df.loc[:, '240RS'] = round((twse_df['價格指數值']/twse_df['240MA'])*100, 4)
twse_df = twse_df.fillna(0)
#twse_df_new = twse_df.iloc[-2:]
twse_df_new = twse_df.loc[twse_df_update.index]  # 只更新尚未計算的新欄位


# 初始化并更新数据库
updater = MySQLUpdater(db_params, 'daily_twse', twse_df_new, '指數名稱', '1')
updater.update_records()
updater.close_connection()

In [8]:
import numpy as np
stock_ids = stock_df_raw['證券代號'].unique().tolist()

stock_df_na = stock_df_raw.isna()
stock_df_update = stock_df_raw[stock_df_na.any(axis=1)] #需要被更新的欄位

for i in stock_ids:
        if((len(i)==4) ): #1. 排除債 ETF等等 2.排除新掛牌資料不到calculate_day天
                #print(i)
                # 計算個股 RS
                temp_df = stock_df_raw[stock_df_raw['證券代號']==i].copy()

                temp_df.loc[:, '5MA'] = temp_df['收盤價'].rolling(5).mean()
                temp_df.loc[:, '8MA'] = temp_df['收盤價'].rolling(8).mean()
                temp_df.loc[:, '10MA'] = temp_df['收盤價'].rolling(10).mean()
                temp_df.loc[:, '20MA']= temp_df['收盤價'].rolling(20).mean()
                temp_df.loc[:, '60MA']= temp_df['收盤價'].rolling(60).mean()
                temp_df.loc[:, '240MA']= temp_df['收盤價'].rolling(240).mean()

                # 避免 temp_df 收盤價為0導致MA計算有誤  
                # if (temp_df['收盤價'] == 0).any() :
                #         temp_df.loc[:, '20RS'] = 0
                #         temp_df.loc[:, '60RS'] = 0
                #         temp_df.loc[:, '240RS'] = 0
                # else :

                temp_df.loc[:, '20RS'] = round((temp_df['收盤價']/temp_df['20MA'])*100, 4)
                temp_df.loc[:, '60RS'] = round((temp_df['收盤價']/temp_df['60MA'])*100, 4)
                temp_df.loc[:, '240RS'] = round((temp_df['收盤價']/temp_df['240MA'])*100, 4)

                # 還是直接算但改用replace改掉 inf
                temp_df.replace([np.inf, -np.inf], 0, inplace=True)     
                temp_df = temp_df.fillna(0)

                #只更新尚未update欄位
                temp_update_df = stock_df_update[stock_df_update['證券代號']==i].copy()
                temp_df_new = temp_df.loc[temp_update_df.index]  # 只更新尚未計算的新欄位
                # 初始化并更新数据库
                updater = MySQLUpdater(db_params, 'daily_price', temp_df_new, '證券代號','1')
                updater.update_records()
updater.close_connection()

In [9]:
# 

import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime, timedelta

# 設置MySQL資料庫連接
db_user = 'root'
db_password = '19970730'
db_host = '127.0.0.1'
db_name = 'sql_stock'

# 創建資料庫連接引擎
engine = create_engine(f'mysql+mysqlconnector://{db_user}:{db_password}@{db_host}/{db_name}')

# 設置日期範圍
#today = datetime(2024, 5, 25)
start_date = pd.to_datetime(stock_df_update['Date']).unique().tolist()[0].date()
end_date = pd.to_datetime(stock_df_update['Date']).unique().tolist()[-1].date()

# 格式化日期
start_date = start_date.strftime('%Y%m%d')
end_date = end_date.strftime('%Y%m%d')

# SQL 查詢 個股
query_stock = f"""
SELECT *
FROM daily_price
WHERE date >= '{start_date}' AND date <= '{end_date}'
"""

stock_df_update_done = pd.read_sql(query_stock, engine)


In [10]:
stock_df_update_done.head(5)

,Date,證券代號,證券名稱,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,...,60MA,60RS,20RS_sd,20RS_rank,60RS_sd,60RS_rank,240MA,240RS,240RS_sd,240RS_rank
0,2025-02-14,0050,元大台灣50,26452171,69782,5136323011,195.15,195.15,193.50,194.00,...,195.2992,99.3348,None,None,None,None,179.2494,108.2291,None,None
1,2025-02-14,0051,元大中型100,72289,378,5653437,78.00,78.40,78.00,78.40,...,77.2700,101.4624,None,None,None,None,79.5631,98.5381,None,None
2,2025-02-14,0052,富邦科技,1413420,3294,274054143,194.55,194.60,193.35,193.65,...,193.4267,100.1155,None,None,None,None,177.0831,109.3554,None,None
3,2025-02-14,0053,元大電子,16788,117,1741306,104.00,104.00,103.25,103.40,...,103.4575,99.9444,None,None,None,None,96.3940,107.2681,None,None
4,2025-02-14,0055,元大MSCI金融,135813,218,3920376,28.88,28.90,28.85,28.86,...,28.3410,101.8313,None,None,None,None,27.5407,104.7903,None,None


In [11]:
twse_df_new

,Date,指數名稱,價格指數值,報酬指數值,漲跌點數,漲跌百分比,5MA,8MA,10MA,20MA,20RS,60MA,60RS,240MA,240RS
26240,2025-02-14,發行量加權股價指數,23152.61,-,246.8,-1.05,23295.592,23304.30125,23192.308,23104.0955,100.21,23008.767167,100.6252,21970.837292,105.3788


In [12]:
# 計算個股對大盤RS
#先轉置取Date
stock_df_update_done['Date'] = pd.to_datetime(stock_df_update_done['Date'])

#stock_date_list = stock_df_raw['Date'].unique().tolist()
stock_date_list =  pd.to_datetime(stock_df_update['Date']).unique().tolist()  # 指查詢要更新的日期

# 要注意一下twse_df_new是不是有資料，沒有的話twse_df_new會空掉就沒辦法更新RS 因為是跟大盤在比 所以要改用註解這段

# twse_df = twse_df_raw[twse_df_raw['指數名稱'] == '發行量加權股價指數'].copy()
# twse_df['Date'] = pd.to_datetime(twse_df['Date'])
# twse_df.rename(columns={'20RS': '20RS_twse','60RS': '60RS_twse','240RS': '240RS_twse'}, inplace=True)
# cols = ['Date', '價格指數值', '20RS_twse', '60RS_twse', '240RS_twse']
# twse_df_forRS = twse_df[cols]


twse_df_new['Date'] = pd.to_datetime(twse_df_new['Date']) 
twse_df_new.rename(columns={'20RS': '20RS_twse','60RS': '60RS_twse','240RS': '240RS_twse'}, inplace=True)
cols = ['Date', '價格指數值', '20RS_twse', '60RS_twse', '240RS_twse']
twse_df_forRS = twse_df_new[cols]

merged_df = pd.merge(stock_df_update_done, twse_df_forRS, on='Date', how='inner')
merged_df['20RS_sd'] = merged_df['20RS']/merged_df['20RS_twse']  #20MA計算RS
merged_df['60RS_sd'] = merged_df['60RS']/merged_df['60RS_twse']  #60MA計算RS
merged_df['240RS_sd'] = merged_df['240RS']/merged_df['240RS_twse']  #240MA計算RS
merged_df = merged_df.fillna(0)

col2 = ['Date', '證券代號', '20RS_sd','20RS_rank','60RS_sd','60RS_rank','240RS_sd','240RS_rank']
for i in range(len(stock_date_list)) :
    temp_merged_df = merged_df[merged_df['Date']==stock_date_list[i]].copy()
    # #標準化成0-100名
    temp_merged_df['20RS_rank'] = (temp_merged_df['20RS_sd'].rank(axis=0,method='first')/len(temp_merged_df))*100
    temp_merged_df['60RS_rank'] = (temp_merged_df['60RS_sd'].rank(axis=0,method='first')/len(temp_merged_df))*100
    temp_merged_df['240RS_rank'] = (temp_merged_df['240RS_sd'].rank(axis=0,method='first')/len(temp_merged_df))*100
    temp_merged_df = temp_merged_df[col2]


    # 初始化并更新数据库
    updater = MySQLUpdater(db_params, 'daily_price', temp_merged_df, '證券代號','2')
    updater.update_records()
updater.close_connection()



# df_rs_week = pd.concat([df_rs_week,df_rs70])
# df_rsall_week = pd.concat([df_rsall_week,df_rs])
# # 順便統計細產業佔比
# rs70_sub_category = get_sub_category_list(sub_category_list,df_rs70,day)                        
# df_rs70_category_week = pd.concat([df_rs70_category_week,rs70_sub_category])


C:\Users\USER\AppData\Local\Temp\ipykernel_22140\109186288.py:26: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged_df = merged_df.fillna(0)


策略一 長期盤整後突破  

濾網
1. 收盤>60日線
2. 60日>240日
3. 成值>1億
4. 成交量>2000張

距離一年內低點20%以上
距離一年內高點20%以內
一年內高低點30%以內
過去一年成交金額小於8000萬
過去一年60RS_RANK 平均小於 50


60RS_RANK 75以上買進 75以下賣出

In [13]:
# 建立連線
# 準備每日個股資料

import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime, timedelta

# 設置MySQL資料庫連接
db_user = 'root'
db_password = '19970730'
db_host = '127.0.0.1'
db_name = 'sql_stock'

# 創建資料庫連接引擎
engine = create_engine(f'mysql+mysqlconnector://{db_user}:{db_password}@{db_host}/{db_name}')

# 設置日期範圍
#today = datetime(2024, 5, 25)
today = datetime.now().date()
one_year_ago = today - timedelta(days=720)

# 格式化日期
today_str = today.strftime('%Y%m%d')
one_year_ago_str = one_year_ago.strftime('%Y%m%d')


# SQL 查詢 個股
query_stock = f"""
SELECT *
FROM daily_price
WHERE date >= '{one_year_ago_str}' AND date <= '{today_str}'
"""


# 使用 pd.read_sql 來執行查詢並讀取數據到 DataFrame

stock_df_raw = pd.read_sql(query_stock, engine)


In [14]:
# 如果是跑昨天 要手動改

# today = datetime.now().date()
# runday_str = today.strftime('%Y-%m-%d')

runday_str

'2025-02-14'

In [15]:
runday_str = '2024-12-11'

In [16]:
import warnings
# 每日跑濾網

stock_df_raw['Date'] = pd.to_datetime(stock_df_raw['Date'])
filter_stock_list = pd.DataFrame(columns=['Date','證券代號','證券名稱','成交股數','成交筆數', '成交金額','開盤價', '收盤價'])
 

stock_df_today = stock_df_raw[stock_df_raw['Date'] == runday_str].copy() 

# 收盤在季線以上 年線以上 成值 1億 
today_list = stock_df_today[(stock_df_today['收盤價']>stock_df_today['60MA']) & 
                        (stock_df_today['60MA']>stock_df_today['240MA']) & 
                        (stock_df_today['成交金額']>100000000) &
                        (stock_df_today['成交股數']>2000000) & 
                        (stock_df_today['60RS_rank']>75) ]

# 計算一年內高點
stock_id_list = today_list['證券代號'].unique().tolist()

stock_id_filter = []
for i in stock_id_list:
        if((len(i)==4) ): #1. 排除債 ETF等等 2.排除新掛牌資料不到calculate_day天

                # 計算個股 RS
                today_df = stock_df_raw[(stock_df_raw['證券代號']==i) & (stock_df_raw['Date'] == runday_str) ].copy()
                close = today_df['收盤價'].iloc[0]

                #print(i)
                # 計算個股 RS
                temp_df_his = stock_df_raw[(stock_df_raw['證券代號']==i) & (stock_df_raw['Date'] < runday_str) & (stock_df_raw['收盤價'] != 0) ].copy()
                # 一年內高低點
                temp_df_his.loc[:, '120DAY_min'] = temp_df_his['收盤價'].rolling(120).min()
                temp_df_his.loc[:, '120DAY_max'] = temp_df_his['收盤價'].rolling(120).max()
                temp_df_his.loc[:, '120DAY_money'] = temp_df_his['成交金額'].rolling(120).mean()


                DAY120_min = temp_df_his.tail(1)['120DAY_min'].iloc[0]
                DAY120_max = temp_df_his.tail(1)['120DAY_max'].iloc[0]


                DAY240_money = temp_df_his.tail(1)['120DAY_money'].iloc[0]
                MA20 = temp_df_his.tail(1)['20MA'].iloc[0] 
                
                # 計算個股 60日RS rank 從原本小於 80 飆上來 
                temp_df_his.loc[:, '60RS_rank_mean'] = temp_df_his['60RS_rank'].rolling(30).mean()
                RS_rank_mean = temp_df_his.tail(1)['60RS_rank_mean'].iloc[0]
                if (DAY120_min == 0) |  (DAY120_max == 0) :
                        print('DAY240_min = 0 stock_name:' + temp_df_his['證券名稱'].iloc[0] + "runday_str" + runday_str + "stock_id" + i)
                else:
                        max_min_diff = abs(DAY120_max - DAY120_min) / DAY120_min 

                        # RS_rank_min = 90
                        #距離一年高點20%以內
                        #距離一年低點20%以上
                        if( (close*1.2 >  DAY120_max) & 
                        (close >  DAY120_min*1.2) & 
                        (RS_rank_mean <  50) & 
                        (DAY240_money < 80000000) & 
                        (close < MA20*1.1) & 
                        (max_min_diff < 0.3)):
                                stock_id_filter.append(i) 


filter_stock_today = stock_df_today[stock_df_today['證券代號'].isin(stock_id_filter)]
filter_stock_today = filter_stock_today[['Date','證券代號','證券名稱','成交股數','成交筆數', '成交金額','開盤價', '收盤價']]
if (len(filter_stock_today)!=0):
        filter_stock_list = pd.concat([filter_stock_list, filter_stock_today], ignore_index=True)


filter_stock_list

# 篩到之後 隔天收盤買
# 當60RS_rank 小於 75 隔天開盤再賣


,Date,證券代號,證券名稱,成交股數,成交筆數,成交金額,開盤價,收盤價


ㄏ 字形股票篩選


第一步 用RS RANK 判斷

參數 
1. RS20_rank_p
2. RS60_rank_p
3. RS240_rank_p

五日線拉平 所以  20RS_RANK  小於 10 
二十日線繼續往上 60RS_RANK  大於 60
六十日線繼續往上 240RS_RANK 大於 80

第二步 判斷均線糾結 
5MA 10MA 20MA糾結

參數 
1. tangled 

糾結 小於等於 0.1

盤整時間 5日線拉平等20日線持續往上至兩線交會
參數

1. relax_day 
2. up_day
3. max_min_diff_p
4. slope_angle5_p

20日斜率  盤整10天
5日斜率   過去30天 需有幾天是斜率陡升
盤整時期高低點落差不要太大 目前是用 盤整最高價 跟 收盤最低價 之前相差 0.35內
因為強勢股希望部要留太多上引線但可以有很多下引線支撐

買法 
1. 撈出股票之後隔天掛前一天收盤價買 (如果開盤開太高就算了不追高)

賣法
1. 跌破20RS RANK確立後第二天開盤賣 
2. 跌破前次盤整區間高點(收盤價)後，收盤賣出


In [17]:
import warnings
import numpy as np
# 計算出場  使用60RS看看
# 忽略 FutureWarning
warnings.simplefilter(action='ignore', category=FutureWarning)

# 設置日期範圍
run_day = datetime.now().date()
month_ago = run_day - timedelta(days=240)

# 格式化日期
run_day = run_day.strftime('%Y-%m-%d')
month_ago = month_ago.strftime('%Y-%m-%d')


# 篩選初始ㄏ字型
RS20_rank_p = 10
RS60_rank_p = 60
RS240_rank_p = 80
# 均線糾結範圍
tangled = 0.1

# 20日線上升天數
relax_day = 10
# 5日線上升天數
up_day = 30
#盤整區間上下震盪差距
max_min_diff_p = 0.35

rs_value = 77
slope = 2.75

profolio_df1 = pd.DataFrame(columns=['RS20_rank_p','RS60_rank_p','slope' ,'relax_day','up_day','rs_value', 'mean_close','mean_close_adj','mean_close_adj_open_af', 'mean_open', 'mean_close_af','mean_open_af','hold_day_mean','hold_day_mean_af','hold_day_mean_adj','hold_day_mean_adj_open_af'])
stock_df_raw['Date'] = pd.to_datetime(stock_df_raw['Date'])


# 想想看要怎麼搞出 ㄏ 字形 參考寶碩 海報哥

# 收盤在季線以上 月線以上 成值 1億 
df_unique0 = stock_df_raw[(stock_df_raw['Date']>= month_ago ) & (stock_df_raw['Date']<= run_day ) & 
                        (stock_df_raw['20RS_rank'] < RS20_rank_p) & 
                        (stock_df_raw['60RS_rank'] > RS60_rank_p) & 
                        (stock_df_raw['240RS_rank'] > RS240_rank_p) &
                        (stock_df_raw['成交金額']>1000000) & 
                        (stock_df_raw['收盤價']> stock_df_raw['60MA']) ].copy()

# 計算各個移動平均線之間的相對差異
df_unique0['diff_10_20'] = np.abs(df_unique0['10MA'] - df_unique0['20MA']) / df_unique0['20MA']
df_unique0['diff_10_5'] = np.abs(df_unique0['10MA'] - df_unique0['5MA']) / df_unique0['5MA']
df_unique0['diff_20_5'] = np.abs(df_unique0['20MA'] - df_unique0['5MA']) / df_unique0['5MA']

# 檢查所有相對差異是否都在10%以內
df_unique0['is_within_10_percent'] = (df_unique0['diff_10_20'] <= tangled) & (df_unique0['diff_10_5'] <= tangled) & (df_unique0['diff_20_5'] <= tangled)

# 篩選出符合條件的列
df_within_10_percent = df_unique0[df_unique0['is_within_10_percent']]


filter_stock_list = pd.DataFrame(columns=['Date','證券代號','證券名稱','成交股數','成交筆數', '成交金額','開盤價', '收盤價'])

# 盤整時間
stock_id_filter = []
collected_data = pd.DataFrame(columns=['證券代號','證券名稱', '收盤價', 'Date', 'relax_day_max','relax_day_close', 'relax_day_min', 'slope20_count','slope5_count'])

for i in range(len(df_within_10_percent)):
        temp_df = df_within_10_percent.iloc[i].copy()

        # 均線糾結那天 
        filter_day = pd.to_datetime(temp_df['Date'])
        # STOCK ID
        filter_stock_id = temp_df['證券代號']

        # ㄏ字形中間盤整區間 高低差不要大於20% max_min_diff < 0.2
        temp_df_his = stock_df_raw[(stock_df_raw['證券代號']==filter_stock_id) & (stock_df_raw['Date'] < filter_day.strftime('%Y-%m-%d')) & (stock_df_raw['收盤價'] != 0) ].copy()
        # 一年內高低點
        temp_df_his.loc[:, 'relax_day_min'] = temp_df_his['收盤價'].rolling(relax_day).min()
        temp_df_his.loc[:, 'relax_day_max'] = temp_df_his['最高價'].rolling(up_day).max()  # 最大值看ㄏ的頭 看長一點 (看要用最高價還是 收盤價)

        temp_df_his.loc[:, 'relax_day_close'] = temp_df_his['收盤價'].rolling(up_day).max()  # 最大值看ㄏ的頭 看長一點 (看要用最高價還是 收盤價)


        relax_day_min = temp_df_his.tail(1)['relax_day_min'].iloc[0]
        relax_day_max = temp_df_his.tail(1)['relax_day_max'].iloc[0]
        relax_day_close = temp_df_his.tail(1)['relax_day_close'].iloc[0]


        if (relax_day_min == 0) |  (relax_day_max == 0) :
                print('relax_day = 0 stock_name:' + temp_df['證券名稱'].iloc[0] + "today_str" + filter_day + "stock_id" + filter_stock_id)
        else:
                max_min_diff = abs(relax_day_min - relax_day_max) / relax_day_min 


        # 計算20日線斜率  ㄏ   
        temp_his_tail20 = temp_df_his.tail(relax_day).copy()
        temp_his_tail20['20MA_Slope'] = temp_his_tail20['20MA'].diff() / temp_his_tail20['20MA'].shift(1)
        # 将斜率转换为角度
        temp_his_tail20['Slope_Angle20'] = np.degrees(np.arctan(temp_his_tail20['20MA_Slope']))
        # 第一天不會有斜率
        temp_his_tail20 = temp_his_tail20.dropna()


        # 計算5日線斜率  ㄏ   
        temp_his_tail5 = temp_df_his.tail(up_day).copy()
        temp_his_tail5['5MA_Slope'] = temp_his_tail5['5MA'].diff() / temp_his_tail5['5MA'].shift(1)
        temp_his_tail5['Slope_Angle5'] = np.degrees(np.arctan(temp_his_tail5['5MA_Slope']))
        # 第一天不會有斜率
        temp_his_tail5 = temp_his_tail5.dropna()


        slope20_count = len(temp_his_tail20[temp_his_tail20['Slope_Angle20'] < 0])
        slope5_count = len(temp_his_tail5[temp_his_tail5['Slope_Angle5'] > slope])  # 看起來大於1就會開始很斜了

        if( (slope20_count == 0) & (slope5_count > 2) & (max_min_diff < max_min_diff_p) &  (temp_df['收盤價'] * 1.2 > relax_day_max) ):
                result_df = pd.DataFrame({
                '證券代號': [ filter_stock_id],
                '證券名稱': [temp_df['證券名稱']],
                '收盤價': [temp_df['收盤價']],
                'Date': [temp_df['Date']],
                'relax_day_max': [relax_day_max],
                'relax_day_close': [relax_day_close],
                'relax_day_min': [relax_day_min],
                'slope20_count': [slope20_count],
                'slope5_count': [slope5_count]
                })
                # 删除全为NA的列，避免可能出现的警告
                result_df.dropna(axis=1, how='all', inplace=True)

                if not result_df.empty:
                        collected_data = pd.concat([collected_data, result_df], ignore_index=True)

                # stock_id_filter.append(filter_stock_id) 


filter_stock_uni = collected_data.drop_duplicates(subset='證券代號', keep='last')

# =======================================================================================================
# 往後篩 突破前高追 
collected_data2 = pd.DataFrame(columns=['Date','證券代號','證券名稱','成交股數','成交筆數', '成交金額','開盤價', '收盤價','relax_day_close'])

# relax_day = 20
# stock_id_filter = []

for i in range(len(filter_stock_uni)):
        temp_df = filter_stock_uni.iloc[i].copy()

        # 篩出結果那天
        filter_day = pd.to_datetime(temp_df['Date'])

        # 突破前高需在六個月內
        filter_day2 = filter_day + timedelta(days=180)

        # STOCK ID
        filter_stock_id = temp_df['證券代號']
        # 前高 
        before_max = temp_df['relax_day_max']

        # 名單開始往後看 哪天突破前高
        temp_df_after = stock_df_raw[(stock_df_raw['證券代號']==filter_stock_id) & (stock_df_raw['Date'] > filter_day.strftime('%Y-%m-%d')) & (stock_df_raw['Date'] < filter_day2.strftime('%Y-%m-%d')) & (stock_df_raw['收盤價'] > before_max) & (stock_df_raw['60RS_rank'] > 80)].copy()

        temp_df_after_1 = temp_df_after[['Date','證券代號','證券名稱','成交股數','成交筆數', '成交金額','開盤價', '收盤價']].head(1)

        temp_df_after_1['relax_day_close']  = temp_df['relax_day_close']# 前高收盤價


        if not temp_df_after_1.empty:
                # 產生最後的list 
                collected_data2 = pd.concat([collected_data2, temp_df_after_1], ignore_index=True)


# collected_data3 = pd.DataFrame(columns=['RS20_rank_p','RS60_rank_p','slope' ,'relax_day','up_day','rs_value','stock_id', 'stock_name', 'Date', '收盤價', '隔天買進價', '隔天最低價',
#                                     '收盤價_RS小於','開盤價_RS小於','隔天收盤_RS小於','隔天開盤_RS小於', 
#                                     '收盤價_dff','收盤價_停損dff','開盤價_dff','收盤價_隔天收盤_dff','收盤價_隔天開盤_dff','diff_adj','diff_adj_open_af','持有天數','持有天數_af','持有天數_停損','持有天數_adj','持有天數_adj_open_af'])

# for i in range(len(collected_data2)):
#     temp_df = collected_data2.iloc[i].copy()

#     filter_day = pd.to_datetime(temp_df['Date'])
#     # filter_day30 = filter_day + pd.Timedelta(days=hold_day)
#     filter_stock_id = temp_df['證券代號']

#     # 盤整前高
#     before_max_close = temp_df['relax_day_close'] 

#     # 篩到之後 隔天一開盤要買 (回測起來 隔天收盤買效果好像更好ㄟ)

#     tomorrow_close_buy = stock_df_raw[(stock_df_raw['證券代號'] == filter_stock_id) & (stock_df_raw['Date'] > filter_day.strftime('%Y-%m-%d'))]['收盤價'].iloc[0]
#     tomorrow_low_buy = stock_df_raw[(stock_df_raw['證券代號'] == filter_stock_id) & (stock_df_raw['Date'] > filter_day.strftime('%Y-%m-%d'))]['最低價'].iloc[0]

#     # 隔日不追高 如果開盤開太高就算了 開盤直接掛昨天收盤價買買看
#     if tomorrow_low_buy > temp_df['收盤價']:
#         continue
#     else :
#         tomorrow_close_buy = temp_df['收盤價']


#     # tomorrow_open = stock_df_raw[(stock_df_raw['證券代號'] == filter_stock_id) & (stock_df_raw['Date'] > filter_day.strftime('%Y-%m-%d'))]['開盤價'].iloc[0]

#     # 找到跌落RS後 不可能那天開盤價可以賣 要重測一次
#     RS75_df_t = stock_df_raw[(stock_df_raw['證券代號'] == filter_stock_id) & (stock_df_raw['Date'] > filter_day.strftime('%Y-%m-%d')) & (stock_df_raw['20RS_rank'] < rs_value)].copy()
#     # 這天是技術面需要離場的日子
#     getout_day = RS75_df_t['Date'].iloc[0]
#     # 真正能夠離場的日子
#     RS75_df = stock_df_raw[(stock_df_raw['證券代號'] == filter_stock_id) & (stock_df_raw['Date'] > getout_day.strftime('%Y-%m-%d'))].copy()

#     # 突破失敗停損 收盤跌破盤整區間上緣就出場

    
#     stoploss_df = stock_df_raw[(stock_df_raw['證券代號'] == filter_stock_id) & (stock_df_raw['Date'] > filter_day.strftime('%Y-%m-%d')) & (stock_df_raw['收盤價'] < before_max_close)].copy()

#     if RS75_df.empty:
#         continue
    
#     getout_day_df = RS75_df.iloc[0] # 符合條件當天
#     af_getout_day_df = RS75_df.iloc[1] # 符合條件隔天

#     diff_day = (getout_day_df['Date'] - temp_df['Date']).days

#     diff_close = round(100 * (getout_day_df['收盤價'] / tomorrow_close_buy - 1), 2) #撈出來當天收盤賣
#     diff_adj = diff_close
#     diff_adj_day = diff_day

#     diff_open = round(100 * (getout_day_df['開盤價'] / tomorrow_close_buy - 1), 2)  #撈出來當天開盤賣

#     diff_day_af = (af_getout_day_df['Date'] - temp_df['Date']).days

#     diff_close_af = round(100 * (af_getout_day_df['收盤價'] / tomorrow_close_buy - 1), 2) #撈出來隔天收盤賣
#     diff_open_af = round(100 * (af_getout_day_df['開盤價'] / tomorrow_close_buy - 1), 2)  #撈出來隔天開盤賣
#     diff_adj_open_af = diff_open_af     
#     diff_adj_day_open_af = diff_day_af

#     # default
#     diff_day_loss = -100
#     diff_close_loss = -100
#     # 多判斷是否停損不續抱著
#     if not stoploss_df.empty:
#             stoploss_day_df = stoploss_df.iloc[0] # 符合條件當天停損
#             if stoploss_day_df['Date'] < getout_day_df['Date']:
#                 diff_day_loss = (stoploss_day_df['Date'] - temp_df['Date']).days
#                 diff_close_loss = round(100 * (stoploss_day_df['收盤價'] / tomorrow_close_buy - 1), 2) #撈出來當天

#                 diff_adj = diff_close_loss # 調整停損值
#                 diff_adj_day = diff_day_loss
#                 diff_adj_open_af = diff_close_loss
#                 diff_adj_day_open_af = diff_day_loss


#     result_df = pd.DataFrame({
#         'RS20_rank_p': [RS20_rank_p], 
#         'RS60_rank_p': [RS60_rank_p],
#         'slope': [slope],
#         'relax_day': [relax_day],
#         'up_day': [up_day],
#         'rs_value': [rs_value],
#         'stock_id': [filter_stock_id],
#         'stock_name': [temp_df['證券名稱']],
#         'Date': [temp_df['Date']],
#         '收盤價': [temp_df['收盤價']],
#         '隔天買進價': [tomorrow_close_buy],
#         '隔天最低價': [tomorrow_low_buy],
#         '收盤價_RS小於': [getout_day_df['收盤價']],
#         '開盤價_RS小於': [getout_day_df['開盤價']],
#         '隔天收盤_RS小於': [af_getout_day_df['收盤價']],
#         '隔天開盤_RS小於': [af_getout_day_df['開盤價']],
#         '收盤價_dff': [diff_close],
#         '收盤價_停損dff': [diff_close_loss],
#         '開盤價_dff': [diff_open],
#         '收盤價_隔天收盤_dff': [diff_close_af],
#         '收盤價_隔天開盤_dff': [diff_open_af],
#         'diff_adj': [diff_adj],
#         'diff_adj_open_af': [diff_adj_open_af],
#         '持有天數': [diff_day],
#         '持有天數_af': [diff_day_af],
#         '持有天數_停損': [diff_day_loss],
#         '持有天數_adj': [diff_adj_day],
#         '持有天數_adj_open_af': [diff_adj_day_open_af],
#     })

#     # 删除全为NA的列，避免可能出现的警告
#     result_df.dropna(axis=1, how='all', inplace=True)

#     if not result_df.empty:
#         collected_data3 = pd.concat([collected_data3, result_df], ignore_index=True)

# # 仅当collected_data不为空时，计算mean并合并到profolio_df中
# if not collected_data3.empty:
#     mean_close = collected_data3['收盤價_dff'].mean()
#     mean_close_adj = collected_data3['diff_adj'].mean()
#     mean_close_adj_open_af = collected_data3['diff_adj_open_af'].mean()
#     mean_open = collected_data3['開盤價_dff'].mean()
#     mean_close_af = collected_data3['收盤價_隔天收盤_dff'].mean()
#     mean_open_af = collected_data3['收盤價_隔天開盤_dff'].mean()
#     hold_day_mean = collected_data3['持有天數'].mean()
#     hold_day_mean_af = collected_data3['持有天數_af'].mean()
#     hold_day_mean_adj = collected_data3['持有天數_adj'].mean()
#     hold_day_mean_adj_open_af = collected_data3['持有天數_adj_open_af'].mean()
#     profolio_data = pd.DataFrame({
#         'RS20_rank_p': [RS20_rank_p], 
#         'RS60_rank_p': [RS60_rank_p],
#         'slope': [slope],
#         'relax_day': [relax_day],
#         'up_day': [up_day],
#         'rs_value': [rs_value],
#         'mean_close': [mean_close],
#         'mean_close_adj': [mean_close_adj],
#         'mean_close_adj_open_af': [mean_close_adj_open_af],
#         'mean_open': [mean_open],
#         'mean_close_af': [mean_close_af],
#         'mean_open_af': [mean_open_af],
#         'hold_day_mean' : [hold_day_mean],
#         'hold_day_mean_af' : [hold_day_mean_af],
#         'hold_day_mean_adj' : [hold_day_mean_adj],
#         'hold_day_mean_adj_open_af' : [hold_day_mean_adj_open_af]
#     })
#     profolio_df1 = pd.concat([profolio_df1, profolio_data], ignore_index=True)


# profolio_df1['pen_close'] = profolio_df1['mean_close']/ profolio_df1['hold_day_mean']  
# profolio_df1['pen_close_adj'] = profolio_df1['mean_close_adj']/ profolio_df1['hold_day_mean_adj']  
# profolio_df1['pen_adj_open_af'] = profolio_df1['mean_close_adj_open_af']/ profolio_df1['hold_day_mean_adj_open_af']  
# profolio_df1['pen_open'] = profolio_df1['mean_open']/ profolio_df1['hold_day_mean']

# profolio_df1['pen_close_af'] = profolio_df1['mean_close_af']/ profolio_df1['hold_day_mean_af']
# profolio_df1['pen_open_af'] = profolio_df1['mean_open_af']/ profolio_df1['hold_day_mean_af']

# profolio_df1['diff_pen_close'] = profolio_df1['mean_close_af'] - profolio_df1['mean_close']
# profolio_df1['diff_pen_open'] = profolio_df1['mean_open_af'] - profolio_df1['mean_open']

# profolio_df1[['RS20_rank_p', 'RS60_rank_p','slope','pen_adj_open_af']]


collected_data2 為篩選出來的個股 看今天有沒有新的個股被篩出來

In [18]:
collected_data2.sort_values(by='Date')

,Date,證券代號,證券名稱,成交股數,成交筆數,成交金額,開盤價,收盤價,relax_day_close
1,2024-07-16,4566,時碩工業,21847545,12902,2222947724,94.6,104.5,99.40
0,2024-07-17,4903,聯光通,5524895,3001,156202095,27.4,29.3,26.95
2,2024-10-24,6199,天品,364407,496,25588894,66.0,71.5,62.70
3,2024-12-13,8080,印鉐,889945,806,37236915,42.0,42.0,39.00
4,2025-01-06,8937,合騏,3758902,3599,605956038,155.0,162.5,155.50


符合條件者 
隔天收盤價買進
起算17天后收盤賣出

停損 : 出場看跌破60RS_RANK 88 隔天開盤直接賣出 

參數配合大量(最近好噴) 改成 ['20RS_rank'] > 52

In [19]:
run_day = datetime.now().date()
month_ago = run_day - timedelta(days=30)

# 格式化日期
run_day = run_day.strftime('%Y-%m-%d')
month_ago = month_ago.strftime('%Y-%m-%d')
# 短波強勢股
stock_date_list = stock_df_raw[(stock_df_raw['Date']>=month_ago ) & (stock_df_raw['Date']<= run_day) & (stock_df_raw['證券代號'] == '2330') ]['Date']
collected_data = pd.DataFrame(columns=['證券代號','證券名稱', '收盤價', 'Date', 'VOL', 'VOL20'])

pre_day = ""
rs20_rank_his = 100
# 檢查前波高低點差距
backday = 120

for day in stock_date_list:

        if(pre_day == ""):
               pre_day =  day
               continue

        stock_df_today = stock_df_raw[(stock_df_raw['Date'] == day) &  (stock_df_raw['20RS_rank'] > 52) & (stock_df_raw['60RS_rank'] > 90) & (stock_df_raw['240RS_rank'] > 90) & (stock_df_raw['收盤價'] > stock_df_raw['10MA'] )].copy() 
        stock_id_list = stock_df_today['證券代號'].tolist()

        for stock_id in stock_id_list:

                # 查前一天的 20RS_rank
                stock_his_df = stock_df_raw[(stock_df_raw['Date'] == pre_day) & (stock_df_raw['證券代號'] == stock_id )]
                # 會部會遇到剛好原本很弱 停牌隔天爆噴的可能
                if not stock_his_df.empty:
                        rs20_rank_his = stock_his_df['20RS_rank'].iloc[0]
                else:
                        # 停牌的話只能查stock_df_raw往前找了
                        stock_his_df = stock_df_raw[(stock_df_raw['Date'] < pre_day) & (stock_df_raw['證券代號'] == stock_id )]
                        if not stock_his_df.empty:
                                rs20_rank_his = stock_his_df['20RS_rank'].iloc[-1]
                                
                # 過去
                stock_today = stock_df_raw[(stock_df_raw['Date'] == day) & (stock_df_raw['證券代號'] == stock_id )].iloc[0]
                today_close = stock_today['收盤價']
                today_open = stock_today['開盤價']
                stock_name = stock_today['證券名稱']
                stock_volume = stock_today['成交筆數']


                # 確認量價關係
                stock_bf = stock_df_raw[(stock_df_raw['Date'] <= day) &  (stock_df_raw['證券代號'] == str(stock_id))].copy() 
                VOL =   stock_bf['成交筆數'].iloc[-1] 
                VOL20 = stock_bf['成交筆數'].rolling(20).mean().iloc[-1]
                if(VOL > VOL20 ):
                        continue

                # stock_af_volume = stock_df_raw[(stock_df_raw['Date'] > day) & (stock_df_raw['證券代號'] == stock_id )]['成交筆數'].iloc[0]
                # stock_af_price  =  stock_df_raw[(stock_df_raw['Date'] > day) & (stock_df_raw['證券代號'] == stock_id )]['收盤價'].iloc[0]
                # stock_af_price3  =  stock_df_raw[(stock_df_raw['Date'] > day) & (stock_df_raw['證券代號'] == stock_id )]['收盤價'].iloc[3]
                
                if( (rs20_rank_his < 15) & (stock_volume > 1000) & (today_open <= today_close)):

                        # 檢查前波高低點
                        df_his_min_price = stock_df_raw[(stock_df_raw['證券代號'] == str(stock_id)) & (stock_df_raw['Date'] < day )]['收盤價'].rolling(backday).min().iloc[-1]
                        df_his_max_price = stock_df_raw[(stock_df_raw['證券代號'] == str(stock_id)) & (stock_df_raw['Date'] < day )]['收盤價'].rolling(backday).max().iloc[-1]
                        
                        if (df_his_min_price != 0) and (df_his_max_price != 0):

                                if (today_close / df_his_min_price > 3) or (df_his_max_price / today_close > 1.25):
                                        continue
                                
                        result_df = pd.DataFrame({
                                '證券代號': [ stock_id],
                                '證券名稱': [stock_name],
                                '收盤價': [today_close],
                                'Date': [day],
                                'VOL': [VOL],
                                'VOL20': [VOL20]
                                })
                        # 删除全为NA的列，避免可能出现的警告
                        result_df.dropna(axis=1, how='all', inplace=True)

                        if not result_df.empty:
                                collected_data = pd.concat([collected_data, result_df], ignore_index=True)

        # 更新pre day
        pre_day =  day

collected_data


,證券代號,證券名稱,收盤價,Date,VOL,VOL20
0,3211,順達,204.5,2025-01-22,14562,15961.65
1,4583,台灣精銳,850.0,2025-02-05,2662,3620.45


處置股策略找尋

In [20]:
import re
import requests
import pandas as pd

def convert_to_gregorian1(roc_date):
    """
    Convert a ROC date string (民國年) to Gregorian date string (西元年) in format YYYY-MM-DD.
    Args:
        roc_date (str): Date in ROC format (e.g., "1131216").
    Returns:
        str: Date in Gregorian format (e.g., "2024-12-16").
    """
    year = int(roc_date[:3]) + 1911
    month = roc_date[3:5]
    day = roc_date[5:7]
    return f"{year}-{month}-{day}"

def convert_to_gregorian2(roc_date):
    """
    Convert a ROC date string (民國年) to Gregorian date string (西元年) in format YYYY-MM-DD.
    Args:
        roc_date (str): Date in ROC format (e.g., "1131216").
    Returns:
        str: Date in Gregorian format (e.g., "2024-12-16").
    """
    year, month, day = map(int, roc_date.split("/"))
    year += 1911
    return f"{year}-{month}-{day}"

def fetch_api_data(api_url):
    """Fetch data from a given API URL."""
    try:
        response = requests.get(api_url)
        if response.status_code == 200:
            return response.json()  # Parse JSON response
        else:
            print(f"Failed to fetch data from {api_url}. HTTP Status Code: {response.status_code}")
            return []
    except Exception as e:
        print(f"An error occurred: {e}")
        return []
    

def process_api1_data(api1_data):
    """
    Process api1_data to:
    - Rename columns
    - Extract and split period into str_date and end_date
    - Extract numbers 20 and 5 from DisposalCondition
    """
    processed_data = []
    for record in api1_data:
        # Split and convert period
        period = record["DispositionPeriod"]
        str_date, end_date = period.split("~")
        str_date = convert_to_gregorian1(str_date)
        end_date = convert_to_gregorian1(end_date)

        # Extract specific pattern for "約每XX分鐘撮合一次"
        condition = record["DisposalCondition"]
        # if not re.search(r"約每5分鐘撮合一次", condition):
        #     continue  # Skip records without "約每5分鐘撮合一次"
        match = re.findall(r"約每(20|5)分鐘撮合一次", condition)
        filtered_condition = ",".join(match) if match else None

        # Append processed record
        processed_data.append({
            'Date': convert_to_gregorian1(record["Date"]),
            "證券代號": record["SecuritiesCompanyCode"],
            "stock_name": record["CompanyName"],
            "str_date": str_date,
            "end_date": end_date,
            "condition": filtered_condition,
        })
    return processed_data

def process_api2_data(api1_data):
    """
    Process api2_data to:
    - Rename columns
    - Extract and split period into str_date and end_date
    - Extract numbers 20 and 5 from DisposalCondition
    """
    processed_data = []
    for record in api1_data:
        # Split and convert period
        period = record["DispositionPeriod"]
        str_date, end_date = period.split("～")
        str_date = convert_to_gregorian2(str_date)
        end_date = convert_to_gregorian2(end_date)

        # Extract specific pattern for "約每XX分鐘撮合一次"
        condition = record["Detail"]
        # if not re.search(r"約每五分鐘撮合一次", condition):
        #     continue  # Skip records without "約每5分鐘撮合一次"

        match = re.findall(r"約每(二十|五)分鐘撮合一次", condition)
        filtered_condition = ",".join(match) if match else None
        turn_number = ''
        if filtered_condition == '五':
            turn_number = '5'
        elif filtered_condition == '二十':
            turn_number = '20'
        else :
            turn_number = None
        # Append processed record
        processed_data.append({
            'Date': convert_to_gregorian1(record["Date"]),
            "證券代號": record["Code"],
            "stock_name": record["Name"],
            "str_date": str_date,
            "end_date": end_date,
            "condition": turn_number,
        })
    return processed_data

# Main function to process both API data and combine results
def process_and_combine_data(api1_url, api2_url):
    # Fetch data from both APIs
    api1_data = fetch_api_data(api1_url)
    api2_data = fetch_api_data(api2_url)

    if api1_data and api2_data:
        # Process data from both APIs
        processed_api1_data = process_api1_data(api1_data)
        processed_api2_data = process_api2_data(api2_data)

        # Combine the two lists of processed data
        combined_data = processed_api1_data + processed_api2_data

        # Convert to DataFrame
        df = pd.DataFrame(combined_data)

        # Show the first few rows of the combined DataFrame
        return df

    else:
        print("Failed to fetch or process data from APIs.")

# Example usage
if __name__ == "__main__":
    api1_url = "https://www.tpex.org.tw/openapi/v1/tpex_disposal_information"  # Example API 1 URL
    api2_url = "https://openapi.twse.com.tw/v1/announcement/punish"  # Example API 2 URL

    # Process and combine data from both APIs
    punish_df = process_and_combine_data(api1_url, api2_url)


from datetime import datetime, timedelta
today = datetime.now().date()
runday_str = today.strftime('%Y-%m-%d')

filtered_df = punish_df[
    (punish_df['end_date'] >= runday_str) &
    (punish_df['condition'] == '5') &
    (punish_df['證券代號'].str.len() == 4)
]


# 檢查同一個 stock_id 是否存在其他不同的 condition
valid_stock_ids = punish_df.groupby('證券代號')['condition'].nunique()
valid_stock_ids = valid_stock_ids[valid_stock_ids == 1].index  # 只保留 condition 唯一的 stock_id

# 最後篩選出符合條件的資料
filtered_df = filtered_df[filtered_df['證券代號'].isin(valid_stock_ids)]

filtered_df['Date'] = pd.to_datetime(filtered_df['Date'])
result_df = pd.merge(filtered_df, stock_df_raw, on=['Date', '證券代號'], how='inner')
filter_new_df = result_df[(result_df['20RS_rank'] > 20) & ((result_df['收盤價'] - result_df['開盤價']) > 0) & (result_df['收盤價'] > 10) & (result_df['成交筆數'] > 1000)]
filter_new_ma5_df = filter_new_df[((filter_new_df['收盤價'] / filter_new_df['5MA']) - 1 > 0.05) & ((filter_new_df['收盤價'] / filter_new_df['5MA']) - 1 < 0.15)]

punish_buy_list = filtered_df[filtered_df['證券代號'].isin(filter_new_ma5_df['證券代號'])]


In [21]:
punish_buy_list

,Date,證券代號,stock_name,str_date,end_date,condition


每日成交值前20名 + 濾網股(20RS RANK 要 > 90)

In [22]:
import pandas as pd

run_day = datetime.now().date()
month_ago = run_day - timedelta(days=30)

# 格式化日期
run_day = run_day.strftime('%Y-%m-%d')
month_ago = month_ago.strftime('%Y-%m-%d')

# 篩選指定日期範圍的資料
stock_date_list = stock_df_raw[
    (stock_df_raw['Date'] >= month_ago) & 
    (stock_df_raw['Date'] <= run_day) & 
    (stock_df_raw['證券代號'] == '2330')
]['Date'].unique()

# 初始化 DataFrame 紀錄每日變動
change_log = pd.DataFrame(columns=['Date', '新增股票', '被踢出股票', '濾網股票'])

# 初始化前一天排行榜的股票列表
previous_top_stocks = set()

for idx, day in enumerate(stock_date_list):
    # 當日排行榜股票（Top 30，依成交金額排序）
    stock_df_today = stock_df_raw[stock_df_raw['Date'] == day].sort_values(by='成交金額', ascending=False).head(20)
    # stock_df_today_filter = stock_df_today[(stock_df_today['20RS_rank'] > 90) ]
    current_top_stocks = set(stock_df_today['證券名稱'].values)

    # 新增濾網
    stock_df_today_filter = stock_df_today[(stock_df_today['20RS_rank'] > 90) & (stock_df_today['收盤價'] > stock_df_today['開盤價'])  ]

    if idx > 0:  # 從第二天開始記錄變化
        # 計算進入和被踢出的股票
        new_entries = current_top_stocks - previous_top_stocks  # 當天新增進入的股票
        removed_entries = previous_top_stocks - current_top_stocks  # 當天被踢出的股票

        # 新增條件的股票
        filter_stock = set(stock_df_today_filter['證券名稱'].values)


        # 將變化紀錄到 DataFrame
        change_log = pd.concat([
            change_log,
            pd.DataFrame({
                'Date': [day],
                '新增股票': [', '.join(new_entries)],
                '被踢出股票': [', '.join(removed_entries)],
                '濾網股票' : [', '.join(new_entries & filter_stock)],
            })
        ], ignore_index=True)
    
    # 更新前一天的排行榜
    previous_top_stocks = current_top_stocks

# 查看結果
change_log

,Date,新增股票,被踢出股票,濾網股票
0,2025-01-20,"AES-KY, 新盛力, 立隆電, 緯創, 達邁, 順達, 創意","華星光, 精材, 元大高股息, 萬潤, 亞光, 波若威, 環宇-KY","AES-KY, 新盛力, 立隆電, 緯創, 達邁"
1,2025-01-21,"華星光, 波若威, 環宇-KY, 台光電","達邁, 順達, 創意, 新應材","華星光, 波若威, 環宇-KY"
2,2025-01-22,"智邦, 神達, 聯電, 華城, 台達電","AES-KY, 和椿, 聯亞, 立隆電, 台光電","華城, 神達"
3,2025-02-03,"元大高股息, 大立光, 欣興, 日月光投控, 華碩, 零壹, 元大台灣50, 力積電","奇鋐, 新盛力, 興能高, 緯穎, 華城, 波若威, 環宇-KY, 宏達電",力積電
4,2025-02-04,"長榮, AES-KY, 奇鋐, 台光電, 上銀, 緯穎, 創意","元大高股息, 大立光, 欣興, 日月光投控, 聯電, 力積電, 台達電",
5,2025-02-05,"新日興, 邁達特, 亞光, 瑞昱, 國巨, 健策","AES-KY, 華碩, 零壹, 緯創, 台光電, 元大台灣50",新日興
6,2025-02-06,"和椿, 大銀微系統, 貿聯-KY, 台光電, 欣銓, 台達電","長榮, 智邦, 邁達特, 神達, 創意, 健策","和椿, 大銀微系統, 欣銓"
7,2025-02-07,"富喬, 信驊, 祥碩, 邁達特, 零壹, 環宇-KY, 達邁, 神達, 創意","奇鋐, 大銀微系統, 瑞昱, 貿聯-KY, 國巨, 台光電, 欣銓, 上銀, 台達電","富喬, 祥碩, 邁達特, 零壹, 環宇-KY, 神達"
8,2025-02-10,"大成鋼, 奇鋐, 智邦, 元大台灣50, 健策","新日興, 信驊, 祥碩, 邁達特, 緯穎",大成鋼
9,2025-02-11,"皇昌, 上銀, 大銀微系統, 貿聯-KY, 緯穎, 上詮","智邦, 零壹, 元大台灣50, 達邁, 創意, 健策","貿聯-KY, 皇昌, 上詮, 上銀"
